<a href="https://colab.research.google.com/github/phanmanhtung/Natural-Language-Processing-Course/blob/master/Bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/ggcolab/'  #change dir to your project folder

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!rm -f wiki-en-train.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
    
!rm -f wiki-en-test.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word

!rm -f 02-train-input.txt
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-input.txt

!rm -f 02-train-answer.txt
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-answer.txt

--2020-02-06 13:07:59--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203886 (199K) [text/plain]
Saving to: ‘wiki-en-train.word’

wiki-en-train.word  100%[===================>] 199.11K  --.-KB/s    in 0.01s   

2020-02-06 13:07:59 (14.9 MB/s) - ‘wiki-en-train.word’ saved [203886/203886]

--2020-02-06 13:08:08--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26989 (26K) [t

In [0]:
from collections import defaultdict

# UNIGRAM
def train_bigram(train_file, model_file):
    """Train trigram language model and save to model file
    """
    counts = defaultdict(int)  # count the n-gram (a map count)
    total_count = 0
    #context_counts = defaultdict(int)   # count the context
    with open(train_file) as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')

            for word in words:
              counts[word] += 1
              total_count +=1
              pass # do nothing

    # Save probabilities to the model file            
    with open(model_file, 'w') as fo:
        for ngram, count in counts.items():

            fo.write((ngram + "\t" + str(counts[ngram]/total_count))+ "\n")

            pass

In [0]:
train_bigram('02-train-input.txt', 'train-answer.txt')

In [0]:
!cat train-answer.txt

a	0.25
b	0.25
c	0.125
</s>	0.25
d	0.125


In [0]:
from collections import defaultdict

# BIGRAM
def train_bigram(train_file, model_file):
    """Train trigram language model and save to model file
    """
    counts = defaultdict(int)  # count the n-gram (a map count)
    total_count = 0
    context_counts = defaultdict(int)
    #context_counts = defaultdict(int)   # count the context
    with open(train_file) as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')

            for i in range(1, len(words)):
                # Add bigram and bigram context
                counts[words[i-1]+ " "+ words[i]] += 1             
                context_counts[words[i-1]] += 1

                # Add unigram and unigram context
                total_count += 1
                counts[words[i]] += 1
                context_counts[""] += 1
                pass # do nothing

    # Save probabilities to the model file            
    with open(model_file, 'w') as fo:
        for ngram, count in counts.items():
            # split ngram into an array of words
            ngram_split = ngram.split()

            # BIGRAM
            if len(ngram_split) > 1: # 
              fo.write((ngram + "\t" + str(counts[ngram]/context_counts[ngram_split[0]]))+"\n")

            # UNIGRAM
            else:
              fo.write((ngram + "\t" + str(counts[ngram]/total_count))+"\n")

            pass

In [0]:
train_bigram('02-train-input.txt', 'train-answer.txt')

In [0]:
!cat train-answer.txt

<s> a	1.0
a	0.25
a b	1.0
b	0.25
b c	0.5
c	0.125
c </s>	1.0
</s>	0.25
b d	0.5
d	0.125
d </s>	1.0


In [0]:
def load_bigram_model(model_file):
    """Load the model file

    Args:
        model_file (str): Path to the model file
    
    Returns:
        probs (dict): Dictionary object that map from ngrams to their probabilities
    """
    probs = {}
    with open(model_file, 'r') as f:
        for line in f:
            element = line.split("\t")
            probs[element[0]] = float(element[1])
            pass
    return probs

In [0]:
probs = load_bigram_model('train-answer.txt')
probs

{'</s>': 0.25,
 '<s> a': 0.25,
 'a': 0.25,
 'a b': 0.25,
 'b': 0.25,
 'b c': 0.125,
 'b d': 0.125,
 'c': 0.125,
 'c </s>': 0.125,
 'd': 0.125,
 'd </s>': 0.125}

In [0]:
!cat 02-train-answer.txt

</s>	0.250000
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
b d	0.500000
c	0.125000
c </s>	1.000000
d	0.125000
d </s>	1.000000


In [0]:
print(probs["a b"])
print(probs.get("a b"))

KeyError: ignored

In [0]:
train_bigram('wiki-en-train.word', 'bigram_model.txt')

In [0]:
probs = load_bigram_model('bigram_model.txt')

In [0]:
import math


def test_bigram(test_file, model_file, lambda2=0.95, lambda1=0.95, N=1000000):
    W = 0 # Total word count
    H = 0
    probs = load_bigram_model(model_file)
    with open(test_file, 'r') as f:
        for line in f:
            # get rid of the whitespace
            line = line.strip()
            if line == '':
                continue

            # split line into array of words
            words = line.split()

            # add to the end and beginning
            words.append('</s>')
            words.insert(0, '<s>')
            
            for i in range(1, len(words)):  # Note: starting at 1, after <s>
                # TODO: Write code to calculate smoothed unigram probabilties
                # and smoothed bigram probabilities
                # You should use calculate p1 as smoothed unigram probability
                # and p2 as smoothed bigram probability

                # If the unigram exists in the dictionary
                if (probs.get(words[i])):
                  p1 = lambda1 * probs.get(words[i]) + (1-lambda1) * (1/N)
                else: # We assume the value of probs.get(words[i]) to be 0
                  p1 = lambda1 * 0 + (1-lambda1) * (1/N) 
                
                # If the bigram exists in the dictionary
                if (probs.get(words[i-1] + " " + words[i])):
                  p2 = lambda2 * probs.get(words[i-1] + " " + words[i])+ (1-lambda2) * p1
                else: # We assume the value of probs.get(words[i-1] + " " + words[i]) to be 0
                  p2 = lambda2 * 0 + (1-lambda2) * p1

                W += 1  # Count the words
                H += -math.log2(p2) # We use logarithm to avoid underflow
    H = H/W
    P = 2**H
    
    print("Entropy: {}".format(H))
    print("Perplexity: {}".format(P))

    return P


# So the error occur when there is no "In computational" in the testing set, which appear None type in the probs dictionary
# The solution is to quantify the missing value

In [0]:
train_bigram('wiki-en-train.word', 'bigram_model.txt')
test_bigram('wiki-en-test.word', 'bigram_model.txt')

Entropy: 11.284859117885485
Perplexity: 2495.0605603552463


2495.0605603552463

In [0]:
import nltk
nltk.download('punkt')

with open("/content/wiki-en-test.word", "r") as f:
  sample_text = f.read()

sample_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"In computational linguistics , word-sense disambiguation -LRB- WSD -RRB- is an open problem of natural language processing , which governs the process of identifying which sense of a word -LRB- i.e. meaning -RRB- is used in a sentence , when the word has multiple meanings -LRB- polysemy -RRB- .\nThe solution to this problem impacts other computer-related writing , such as discourse , improving relevance of search engines , anaphora resolution , coherence , inference et cetera .\nResearch has progressed steadily to the point where WSD systems achieve sufficiently high levels of accuracy on a variety of word types and ambiguities .\nA rich variety of techniques have been researched , from dictionary-based methods that use the knowledge encoded in lexical resources , to supervised machine learning methods in which a classifier is trained for each distinct word on a corpus of manually sense-annotated examples , to completely unsupervised methods that cluster occurrences of words , thereby

In [0]:
# Remove all the punctuation (might not use)
# This function is not working, but the idea of dealing with text is this!

import regex as re
def remove_punctuation(words):
  new_words = []
  for word in words:
    new_word = re.sub(r'["\w\s]','', word)
    if new_word != '':
      new_words.append(new_word)

  return new_words

remove_punctuation(sample_text)

In [0]:
# Tokenization: seperate the words into a list

sample_text = sample_text.lower()
tokens = nltk.word_tokenize(sample_text)
print(tokens)

['in', 'computational', 'linguistics', ',', 'word-sense', 'disambiguation', '-lrb-', 'wsd', '-rrb-', 'is', 'an', 'open', 'problem', 'of', 'natural', 'language', 'processing', ',', 'which', 'governs', 'the', 'process', 'of', 'identifying', 'which', 'sense', 'of', 'a', 'word', '-lrb-', 'i.e', '.', 'meaning', '-rrb-', 'is', 'used', 'in', 'a', 'sentence', ',', 'when', 'the', 'word', 'has', 'multiple', 'meanings', '-lrb-', 'polysemy', '-rrb-', '.', 'the', 'solution', 'to', 'this', 'problem', 'impacts', 'other', 'computer-related', 'writing', ',', 'such', 'as', 'discourse', ',', 'improving', 'relevance', 'of', 'search', 'engines', ',', 'anaphora', 'resolution', ',', 'coherence', ',', 'inference', 'et', 'cetera', '.', 'research', 'has', 'progressed', 'steadily', 'to', 'the', 'point', 'where', 'wsd', 'systems', 'achieve', 'sufficiently', 'high', 'levels', 'of', 'accuracy', 'on', 'a', 'variety', 'of', 'word', 'types', 'and', 'ambiguities', '.', 'a', 'rich', 'variety', 'of', 'techniques', 'have'

In [0]:
# count the number of words
token_freq = nltk.FreqDist(tokens)
token_freq

FreqDist({'in': 102,
          'computational': 8,
          'linguistics': 4,
          ',': 233,
          'word-sense': 2,
          'disambiguation': 10,
          '-lrb-': 47,
          'wsd': 32,
          '-rrb-': 47,
          'is': 77,
          'an': 12,
          'open': 1,
          'problem': 12,
          'of': 155,
          'natural': 2,
          'language': 10,
          'processing': 3,
          'which': 22,
          'governs': 1,
          'the': 215,
          'process': 3,
          'identifying': 1,
          'sense': 51,
          'a': 96,
          'word': 60,
          'i.e': 2,
          '.': 180,
          'meaning': 8,
          'used': 20,
          'sentence': 3,
          'when': 3,
          'has': 16,
          'multiple': 1,
          'meanings': 4,
          'polysemy': 3,
          'solution': 3,
          'to': 102,
          'this': 9,
          'impacts': 1,
          'other': 14,
          'computer-related': 1,
          'writing': 2,
       

In [0]:
# Generate bigram

#bigram = nltk.bigrams(tokens)
bigram = nltk.ngrams(tokens, 2) # n=2 is the number of n in n_gram

token_freq = nltk.FreqDist(bigram)
for token in token_freq.keys():
  print(token,token_freq[token])

('in', 'computational') 4
('computational', 'linguistics') 4
('linguistics', ',') 3
(',', 'word-sense') 1
('word-sense', 'disambiguation') 1
('disambiguation', '-lrb-') 1
('-lrb-', 'wsd') 1
('wsd', '-rrb-') 1
('-rrb-', 'is') 5
('is', 'an') 2
('an', 'open') 1
('open', 'problem') 1
('problem', 'of') 2
('of', 'natural') 1
('natural', 'language') 2
('language', 'processing') 2
('processing', ',') 2
(',', 'which') 8
('which', 'governs') 1
('governs', 'the') 1
('the', 'process') 2
('process', 'of') 1
('of', 'identifying') 1
('identifying', 'which') 1
('which', 'sense') 3
('sense', 'of') 3
('of', 'a') 10
('a', 'word') 8
('word', '-lrb-') 2
('-lrb-', 'i.e') 2
('i.e', '.') 2
('.', 'meaning') 1
('meaning', '-rrb-') 1
('is', 'used') 3
('used', 'in') 3
('in', 'a') 9
('a', 'sentence') 1
('sentence', ',') 2
(',', 'when') 2
('when', 'the') 1
('the', 'word') 11
('word', 'has') 1
('has', 'multiple') 1
('multiple', 'meanings') 1
('meanings', '-lrb-') 1
('-lrb-', 'polysemy') 1
('polysemy', '-rrb-') 1
('-

In [0]:
with open("/content/02-train-answer.txt", "r") as f:
  sample_text = f.read()
  print(sample_text)

</s>	0.250000
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
b d	0.500000
c	0.125000
c </s>	1.000000
d	0.125000
d </s>	1.000000



In [0]:
with open("/content/02-train-input.txt", "r") as f:
  sample_text = f.read()

tokens = nltk.word_tokenize(sample_text)
bigram = nltk.ngrams(tokens,   2) # n=2 is the number of n in n_gram

token_freq = nltk.FreqDist(bigram)
for token in token_freq.keys():
  print(token,token_freq[token])

('a', 'b') 2
('b', 'c') 1
('c', 'a') 1
('b', 'd') 1


In [0]:
with open("/content/02-train-input.txt", "r") as f:
  sample_text = f.read()
  text = sample_text.strip()
  words = text.split()
  words.append('</s>')
  words.insert(0, '<s>')

print(words)

['<s>', 'a', 'b', 'c', 'a', 'b', 'd', '</s>']


In [0]:
with open("/content/02-train-input.txt") as f:
  for line in f:
    # strip() throws the first white space (if there is none -> do nothing!)
    line = line.strip()
    
    #if line == '':
    #   continue  

    # split create a list from string
    words = line.split()
    words.append('</s>')
    words.insert(0, '<s>')

words

['<s>', 'a', 'b', 'd', '</s>']

In [0]:
test = " Here is the testing string, do everything you want Here, ok???"
test.split()

['Here',
 'is',
 'the',
 'testing',
 'string,',
 'do',
 'everything',
 'you',
 'want',
 'Here,',
 'ok???']

In [0]:
test.split(", ")

[' Here is the testing string', 'do everything you want Here', 'ok???']

In [0]:
test.strip()

'Here is the testing string, do everything you want Here, ok???'

In [0]:
# left side in elements only
test.strip(" Here")

'is the testing string, do everything you want Here, ok???'

In [0]:
new_test = "Hello there!!!"
new_test.strip()

'Hello there!!!'

In [0]:
token_freq

FreqDist({('a', 'b'): 2, ('b', 'c'): 1, ('b', 'd'): 1, ('c', 'a'): 1})

In [0]:
from collections import defaultdict
counts = defaultdict(int)  # count the n-gram
context_counts = defaultdict(int)   # count the context
print(counts)
print(context_counts)

defaultdict(<class 'int'>, {})
defaultdict(<class 'int'>, {})


In [0]:
# NLP course:
# Application: QA, Information Extraction, Sentiment Analysis (positive&negative), Machine Translation

In [0]:
# Regular expression: Regex
import re

txt = "The rain in Spain"

# Sets [ant] (a or n or t)
# [A-Z] from A to Z (capital letter)
# [^a] not a (negation)

x = re.findall("[a-m]", txt)
y = re.findall("[rai]", txt)
z = re.findall("[^h]", txt)
t = re.findall("[^h-z]", txt)

print(x)
print(y)
print(z)
print(t)

['h', 'e', 'a', 'i', 'i', 'a', 'i']
['r', 'a', 'i', 'i', 'a', 'i']
['T', 'e', ' ', 'r', 'a', 'i', 'n', ' ', 'i', 'n', ' ', 'S', 'p', 'a', 'i', 'n']
['T', 'e', ' ', 'a', ' ', ' ', 'S', 'a']


In [0]:
txt2 = "And i saw him. The cat in the hat, the other in there, the blithe one."

print("the with capital T and lower case t: ", re.findall("[Tt]he", txt2))
# "blithe" and "there" is still in count, to get rid of this
print(re.findall("[Tt]he[^A-Za-z]", txt2))

the with capital T and lower case t:  ['The', 'the', 'the', 'the', 'the', 'the', 'the']
['The ', 'the ', 'the ', 'the ', 'the ']


In [0]:
for i in range(1,10):
  print(i)

1
2
3
4
5
6
7
8
9
